In [1]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import weather_api_key
from config import g_key


In [2]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame
# Study data files
weather_meta_data = "my_weather_data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_meta_data)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,0,Coffs Harbour,-30.3000,153.1333,63.45,73,5.35,54,AU,1599436800
1,1,Zacapú,19.8333,-101.7167,66.63,86,1.43,87,MX,1599436800
2,2,Jalu,29.0331,21.5482,77.36,54,12.82,0,LY,1599436800
3,3,Khatanga,71.9667,102.5000,43.38,77,4.09,22,RU,1599436800
4,4,Punta Arenas,-53.1500,-70.9167,42.71,78,7.58,86,CL,1599436800
...,...,...,...,...,...,...,...,...,...,...
536,536,Guthrie,35.8789,-97.4253,89.47,43,14.88,1,US,1599436800
537,537,Allen,41.0706,-85.0144,74.35,67,12.21,87,US,1599436800
538,538,Rongcheng,30.6340,117.8517,76.15,73,2.21,0,CN,1599436800
539,539,Egvekinot,66.3167,-179.1667,44.04,66,2.66,87,RU,1599436800


In [3]:
# Humidity Heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.

humidity = weatherdf["Humidity"].astype(float)
maxhumidity = humidity.max()

locations = weatherdf[["Latitude", "Longitude"]]

In [4]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity,
                                 dissipating=False, 
                                 max_intensity=maxhumidity,
                                 point_radius=1)
fig.add_layer(heat_layer)

plt.savefig("VacationHeatMap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [5]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

narrowed_city_df = weatherdf.loc[(weatherdf["Max Temp"] > 72) & (weatherdf["Max Temp"] < 85) & (weatherdf["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,2,Jalu,29.0331,21.5482,77.36,54,12.82,0,LY,1599436800
1,16,Rikitea,-23.1203,-134.9692,73.56,84,11.25,0,PF,1599436800
2,56,Buraidah,26.3260,43.9750,82.49,19,6.20,0,SA,1599436800
3,72,Marsá Maţrūḩ,31.3525,27.2453,75.16,71,3.85,0,EG,1599436800
4,128,Vila Velha,-20.3297,-40.2925,74.23,83,12.15,0,BR,1599436800


In [6]:
# Hotel Map
#  Store into variable named hotel_df.
#  Add a "Hotel Name" column to the DataFrame.
#  Set parameters to search for hotels within 5000 meters.
#  Hit the Google Places API for each city's coordinates.
#  Store the first Hotel result into the DataFrame.
#  Plot markers on top of the heatmap.

hotellist = []
for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Latitude']
    lng = narrowed_city_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date,Hotel Name
0,2,Jalu,29.0331,21.5482,77.36,54,12.82,0,LY,1599436800,Jalu
1,16,Rikitea,-23.1203,-134.9692,73.56,84,11.25,0,PF,1599436800,Rikitea
2,56,Buraidah,26.3260,43.9750,82.49,19,6.20,0,SA,1599436800,Buraydah
3,72,Marsá Maţrūḩ,31.3525,27.2453,75.16,71,3.85,0,EG,1599436800,Marsa Matruh
4,128,Vila Velha,-20.3297,-40.2925,74.23,83,12.15,0,BR,1599436800,Vila Velha


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]


In [12]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
plt.savefig("HotelMap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>